In [2]:
import json

with open("/Users/frsc/Documents/Projects/open-birdsite-db/data/local_seed/archives/exgenesis/archive.json", "r") as f:
    archive = json.load(f)

print(f"Loaded archive with {len(archive)} keys")


Loaded archive with 9 keys


In [5]:
from datetime import datetime

def parse_twitter_date(date_str):
    return datetime.strptime(date_str, '%a %b %d %H:%M:%S %z %Y')


def patch_tweets_with_note_tweets(note_tweets, tweets):
    patched_tweets = []
    for tweet_obj in tweets:
        tweet = tweet_obj['tweet']
        matching_note_tweet = next((
            nt['noteTweet'] for nt in note_tweets
            if tweet['full_text'].startswith(nt['noteTweet']['core']['text'][:200])
            and abs(
                (datetime.fromisoformat(tweet['created_at']) - 
                 datetime.fromisoformat(nt['noteTweet']['createdAt'])).total_seconds()
            ) < 1
        ), None)
        
        if matching_note_tweet:
            tweet['full_text'] = matching_note_tweet['core']['text']
        
        patched_tweets.append(tweet_obj)
    
    return patched_tweets

patched_tweets = patch_tweets_with_note_tweets(archive.get('note-tweet', []), archive['tweets'])
print(f"Patched {len(patched_tweets)} tweets")

ValueError: Invalid isoformat string: 'Sat Oct 21 08:26:53 +0000 2023'